In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.callbacks import LearningRateScheduler as LRS
from keras.callbacks import ReduceLROnPlateau as RLRP
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

In [8]:
batch_size = 128
epochs = 200
num_classes=10

In [3]:
def configure_callbacks(model_id, kpi_to_monitor='val_accuracy'):
    # Without log/ or models/ subfolder as not possible to access unexisting folders
    # If possible to train with jupyter revise
    name = "cnn"
    log_filename = '%s-%s.log' % (name, model_id)
    csv_logger = CSVLogger(log_filename)
    
    chk_1_model_filename = '%s-%s-{epoch:04d}-{%s:.6f}.h5' % (name, model_id, kpi_to_monitor)
    chk_2_model_filename = '%s-%s.h5' % (name, model_id)
    
    # Save best model fully not only weights after each epoch (period=1) 
    # with best accuracy value (mode=max, save_best_only=True)
    checkpoint1 = ModelCheckpoint(
        chk_1_model_filename,
        monitor=kpi_to_monitor,
        save_best_only=True,
        save_weights_only=False, 
        verbose=1, mode='max', period=1
    )

    checkpoint2 = ModelCheckpoint(
        chk_2_model_filename, 
        monitor=kpi_to_monitor,
        save_best_only=False,
        save_weights_only=False, 
        verbose=1, mode='auto', period=1
    )

    callbacks = [csv_logger, checkpoint1, checkpoint2]
    return callbacks

In [11]:
# Define a learning rate scheduler
def scheduler(epoch):
    if epoch < 25:
        return 0.1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

scheduler_lr = LRS(scheduler)

In [12]:
# Define a learning rate scheduler
def scheduler_2(epoch):
    if epoch < 75:
        return 0.1
    elif epoch < 125:
        return 0.01
    else:
        return 0.001

scheduler_lr_2 = LRS(scheduler_2)

In [4]:
# Define a learning rate scheduler
def scheduler_3(epoch):
    if epoch < 50:
        return 0.1
    elif epoch < 75:
        return 0.01
    elif epoch < 150:
        return 0.001
    else:
        return 0.0005

scheduler_lr_3 = LRS(scheduler_3)

In [14]:
# Define an automatic learning rate scheduler
reduce_lr = RLRP(monitor='val_loss', factor=0.5, cooldown=1,
                 patience=10, min_lr=0.001)

In [15]:
# Data Augmentation with an ImageGenerator
datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True,
                             vertical_flip=False)

In [5]:
# Data Augmentation with an ImageGenerator + more variations
datagen_plus = ImageDataGenerator(width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=False,
                                  zoom_range=[1.0,1.1],
                                  rotation_range=10.0,
                                  shear_range=0.1)

In [9]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training set (50000, 32, 32, 3)
test set (10000, 32, 32, 3)


In [7]:
# Optimizer 
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9)

In [19]:
def basic_training_and_evaluation(model, model_id, scheduler):
    # Compile Model
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train)/batch_size,
                        epochs=epochs,
                        shuffle=True,
                        verbose=1,
                        validation_data=(x_test, y_test),                   
                        callbacks=[configure_callbacks(model_id), scheduler])
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)
    print('Test error:', (1-accuracy))
    
    return model

In [8]:
def advanced_training_and_evaluation(model, model_id, scheduler):
    # Compile Model
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    history = model.fit(datagen_plus.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train)/batch_size,
                        epochs=epochs,
                        shuffle=True,
                        verbose=1,
                        validation_data=(x_test, y_test),                   
                        callbacks=[configure_callbacks(model_id), scheduler])
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)
    print('Test error:', (1-accuracy))
    
    return model

In [4]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGNM(model, filters, ishape=0):
    if (ishape!=0):
        model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
    else:
        model.add(Conv2D(filters, (3, 3), padding='same'))

        model.add(BN())
        model.add(GN(0.3))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
    return model

In [5]:
def CBGN(model, filters, gaussian_noise, ishape=0):
    if (ishape!=0):
        model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
    else:
        model.add(Conv2D(filters, (3, 3), padding='same'))

        model.add(BN())
        model.add(GN(gaussian_noise))
        model.add(Activation('relu'))
        #model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
    return model

## Extra funcitons

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs,
                    shuffle=True,
                    verbose=1,
                    validation_data=(x_test, y_test),                   
                    callbacks=[configure_callbacks(model_id), scheduler_lr])

In [ ]:
history = model.fit(datagen_plus.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs,
                    shuffle=True,
                    verbose=1,
                    validation_data=(x_test, y_test),                   
                    callbacks=[configure_callbacks(model_id), reduce_lr])

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('Test error:', (1-accuracy))

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False)
history = pd.read_csv("model-weights/model.log")
model = load_model("model-weights/model.h5")

## Models

In [11]:
# Model Basic
model = Sequential()

model = CBGNM(model, 32, x_train.shape[1:])
model = CBGNM(model, 64)
model = CBGNM(model, 128)
model = CBGNM(model, 256)
model = CBGNM(model, 512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model_basic = model
model_basic.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise (GaussianNois (None, 32, 32, 64)        0         
_________________________________________________________________
activation (Activation)      (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       7

In [11]:
model = Sequential()

model = CBGN(model, 64, 0.1, x_train.shape[1:])
model = CBGN(model, 128, 0.2)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 256, 0.3)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 512, 0.3)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 512, 0.3)

model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model_cnn = model
model_cnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
gaussian_noise_4 (GaussianNo (None, 32, 32, 128)       0         
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 128)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 256)      

In [12]:
model = Sequential()

model = CBGN(model, 64, 0.1, x_train.shape[1:])
model = CBGN(model, 64, 0.2)
model = CBGN(model, 64, 0.2)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 128, 0.3)
model = CBGN(model, 128, 0.3)
model = CBGN(model, 128, 0.3)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 256, 0.3)
model = CBGN(model, 256, 0.3)
model = CBGN(model, 256, 0.3)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 512, 0.3)
model = CBGN(model, 512, 0.3)
model = CBGN(model, 512, 0.3)

model.add(MaxPooling2D(pool_size=(2, 2)))

model = CBGN(model, 512, 0.3)
model = CBGN(model, 512, 0.3)
model = CBGN(model, 512, 0.3)

model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model_cnn_vgg = model
model_cnn_vgg.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_8 (GaussianNo (None, 32, 32, 64)        0         
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)       

## Training

In [13]:
# Basic and Basic training
model_1 = model_basic
model_1 = advanced_training_and_evaluation(model=model_1,
                                           model_id='model_1',
                                           scheduler=scheduler_lr_3)

Epoch 1/200
390/390 [==============================] - 35s 72ms/step - loss: 3.5898 - accuracy: 0.1646 - val_loss: 2.1274 - val_accuracy: 0.1730

Epoch 00001: val_accuracy improved from -inf to 0.17300, saving model to cnn-model_1-0001-0.173000.h5

Epoch 00001: saving model to cnn-model_1.h5
Epoch 2/200
390/390 [==============================] - 27s 70ms/step - loss: 2.0159 - accuracy: 0.2169 - val_loss: 1.9496 - val_accuracy: 0.2459

Epoch 00002: val_accuracy improved from 0.17300 to 0.24590, saving model to cnn-model_1-0002-0.245900.h5

Epoch 00002: saving model to cnn-model_1.h5
Epoch 3/200
390/390 [==============================] - 27s 69ms/step - loss: 1.8765 - accuracy: 0.2520 - val_loss: 1.9627 - val_accuracy: 0.2400

Epoch 00003: val_accuracy did not improve from 0.24590

Epoch 00003: saving model to cnn-model_1.h5
Epoch 4/200
390/390 [==============================] - 27s 70ms/step - loss: 1.7329 - accuracy: 0.3302 - val_loss: 2.6530 - val_accuracy: 0.1919

Epoch 00004: val_ac

In [15]:
model_2 = model_cnn
model_2 = advanced_training_and_evaluation(model=model_2,
                                           model_id='model_2',
                                           scheduler=scheduler_lr_3)

Epoch 1/200
390/390 [==============================] - 32s 79ms/step - loss: 1.6692 - accuracy: 0.3830 - val_loss: 2.1341 - val_accuracy: 0.4339

Epoch 00001: val_accuracy improved from -inf to 0.43390, saving model to cnn-model_2-0001-0.433900.h5

Epoch 00001: saving model to cnn-model_2.h5
Epoch 2/200
390/390 [==============================] - 30s 76ms/step - loss: 1.1682 - accuracy: 0.5843 - val_loss: 3.0741 - val_accuracy: 0.4101

Epoch 00002: val_accuracy did not improve from 0.43390

Epoch 00002: saving model to cnn-model_2.h5
Epoch 3/200
390/390 [==============================] - 30s 77ms/step - loss: 0.9469 - accuracy: 0.6648 - val_loss: 1.0245 - val_accuracy: 0.6654

Epoch 00003: val_accuracy improved from 0.43390 to 0.66540, saving model to cnn-model_2-0003-0.665400.h5

Epoch 00003: saving model to cnn-model_2.h5
Epoch 4/200
390/390 [==============================] - 30s 77ms/step - loss: 0.8250 - accuracy: 0.7122 - val_loss: 1.6188 - val_accuracy: 0.5863

Epoch 00004: val_ac

In [18]:
model_3 = model_cnn_vgg
model_3 = advanced_training_and_evaluation(model=model_3,
                                           model_id='model_3',
                                           scheduler=scheduler_lr_3)

Epoch 1/200
390/390 [==============================] - 38s 94ms/step - loss: 1.9025 - accuracy: 0.2662 - val_loss: 3.5055 - val_accuracy: 0.3316

Epoch 00001: val_accuracy improved from -inf to 0.33160, saving model to cnn-model_3-0001-0.331600.h5

Epoch 00001: saving model to cnn-model_3.h5
Epoch 2/200
390/390 [==============================] - 36s 93ms/step - loss: 1.3749 - accuracy: 0.4934 - val_loss: 1.4696 - val_accuracy: 0.5375

Epoch 00002: val_accuracy improved from 0.33160 to 0.53750, saving model to cnn-model_3-0002-0.537500.h5

Epoch 00002: saving model to cnn-model_3.h5
Epoch 3/200
390/390 [==============================] - 36s 93ms/step - loss: 1.0811 - accuracy: 0.6136 - val_loss: 1.6047 - val_accuracy: 0.5417

Epoch 00003: val_accuracy improved from 0.53750 to 0.54170, saving model to cnn-model_3-0003-0.541700.h5

Epoch 00003: saving model to cnn-model_3.h5
Epoch 4/200
390/390 [==============================] - 36s 93ms/step - loss: 0.8986 - accuracy: 0.6880 - val_loss: 